In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import seaborn as sns

sns.set_style("darkgrid", {"axes.facecolor": ".95"})

In [ ]:
# Na variável abaixo é atribuido o caminho para a pasta onde estão os dados do coronavírus
folder = "coronavirus_data/"

# A função 'listdir' do módulo 'os' lista todos os arquivos dentro da pasta que passamos como parâmetro 
files = os.listdir(folder)

# A ideia do dicionário abaixo é ser usado na função pd.DataFrame logo abaixo
# Quando passamos um dicionário para a função pd.DataFrame, ele interpreta cada chave do dicionário como uma coluna e...
# ... cada valor de uma chave - que deve ser uma lista ou um vetor numpy - como os valores para aquela coluna.
# Neste caso, só estamos criando a estrutura do DataFrame, ou seja, suas colunas, pois os valores estão vazios, mas serão
# concatenados dinamicamente com o loop abaixo.
general_dict = {
    "date": [],
    "location": [],
    "new_cases": [],
    "new_deaths": [],
    "total_cases": [],
    "total_deaths": [],
}

df = pd.DataFrame(general_dict)

# O loop abaixo itera sobre cada arquivo dentro da variável 'files'
# A ideia é abrir cada um desses arquivos como um dataframe do pandas e concatená-lo ao dataframe que criamos acima,
# que ficará mais extenso a cada iteração.
for file in files:
    # A função 'read_csv' da library 'pandas' recebe como parâmetro o caminho para um arquivo .csv e 
    # o abre como um pandas DataFrame. Neste caso, além de abrir o .csv como pandas dataframe, estamos colocando o dataframe
    # criado dentro da variável part_df - que será atualizada a cada iteração. Note que o parâmetro é uma concatenação de
    # strings, ou seja, a variável que armazena a pasta do arquivo, e o nome do arquivo em si.
    part_df = pd.read_csv(folder + file)
    
    # Na variável abaixo, pegamos o trecho do nome do arquivo que tem o dia relacionado àqueles dados. Além disso,
    # trocamos o caractére '_' por '-'
    date_from_filename = file[-14:-4].replace("_", "-")
    
    # Criamos a coluna 'date' no nosso dataframe, e todos os valores dessa coluna serão iguais a data que foi extraída
    # e atribuída à variável acima.
    part_df["date"] = date_from_filename
    
    # Aqui nós simplesmente reorganizamos as colunas do nosso dataframe.
    part_df = part_df[["date", "location", "new_cases", "new_deaths", "total_cases", "total_deaths"]]
    
    # Enfim, dizemos que nosso novo dataframe df será a concatenação do dataframe df atual + o part_df que carregamos acima.
    df = pd.concat([df, part_df])
    
    # Todo esse processo será repetido para cada arquivo que está na variável 'files'

# Após toda a iteração, nossos índices estarão embaralhos, sempre recomecando do 0 para cada data.
# Então, resetamos o index, dropando o index antigo
df.reset_index(drop=True, inplace=True)

In [ ]:
# Verificando se a estrutura está legal

df.head()

In [ ]:
# Verificando a quantidade de registros, de registros nulos e os tipos de cada coluna.

df.info()

In [ ]:
# Verificamos que as colunas 'new_cases', 'new_deaths', 'total_cases' e 'total_deaths' são floats, e não inteiros.
# Temos que modificá-los, afinal, não existe nenhum caso novo que seja uma fração.

df["new_cases"] = df["new_cases"].astype("Int64")
df["new_deaths"] = df["new_deaths"].astype("Int64")
df["total_cases"] = df["total_cases"].astype("Int64")
df["total_deaths"] = df["total_deaths"].astype("Int64")

In [ ]:
# Verificando se os tipos de dados foram mudados com success

df.info()

### Nas próximas células, verificaremos quais são os países com mais de X infectados na data mais recente.

In [ ]:
# 1st step: Filtrar o dataframe pela data mais recente.
# Armazene o dataframe filtrado à variável 'most_recent_data'

most_recent_data = df[df["date"] == df["date"].max()]

In [ ]:
# Verificamos como está a distribuição de casos totais para a data mais recente

# Abaixo, definimos as dimensões da figura do plot. Gosto de usar 16x8
plt.figure(figsize=(16, 8))

# Abaixo, pegarei a coluna 'total_cases' do dataframe 'most_recent_data' - que foi filtrado acima.
most_recent_total_cases_data = most_recent_data.total_cases

# Utilizarei a função 'hist' do módulo 'plt' para plotar a distribuição do vetor 'most_recent_total_cases_data',
# que foi atribuído acima.
plt.hist(most_recent_total_cases_data, bins=81)

# As variáveis 'xticks' e 'xticklabels' definirão, respectivamente, a posição dos rótulos e os rótulos no eixo X
xticks = np.arange(0, 80000 + 1, 1000)
xticklabels = ["{}K".format(x) if x % 2 == 0 else "" for x in np.arange(0, 80 + 1, 1)]

# Abaixo, utilizaremos os vetores definidos acima para, enfim, configurar os ticks do eixo X no plot.
# O primeiro parâmetro da função deve ser a posição dos ticks, e o segundo os rótulos. Note: ambos devem
# conter o mesmo número de elementos.
plt.xticks(xticks, xticklabels);

### Verificamos que a grande maioria dos países tem de 0 a 1,000 casos totais. Portanto, vamos filtrar os países que tem pelo menos 1,000 casos totais na data mais recente.

In [ ]:
# Armazene na variável 'mt_1k_infected_countries' uma lista com os países com total_cases >= 1,000 
# de covid-19 na data mais recente

mt_1k_infected_df = most_recent_data[most_recent_data["total_cases"] >= 1000].sort_values(by=["total_cases"], ascending=False)
mt_1k_infected_countries = mt_1k_infected_df.location.unique().tolist()

mt_1k_infected_countries

In [ ]:
# Como ainda tem uma quantidade de países que pode poluir muito nosso plot,
# Vamos selecionar os países com os maiores total_cases, além do Brazil
# Note que, na célula acima, eu ordenei o dataframe filtrado por "total_cases" de forma descrecente. Então, 
# sei que a lista acima está em ordem decrescente de casos totais.

filtered_countries = [
    "China",
    "Italy",
    "United States",
    "Spain",
    "Brazil"
]

In [ ]:
# Filtrando o DataFrame original, ou seja, a variável 'df', pelos países que me interessam.
# Note: utilizei o método .isin() que aceita como parâmetro um iterável. Sendo assim, filtraremos o dataframe 
# original para aqueles países que estão no iterável 'filtered_countries'
mt_1k_infected_master_df = df[df["location"].isin(filtered_countries)]

# Eu desejo ver a evolução temporal dos casos de covid-19 nos países a partir do dia em que eles tiveram 50 casos.
# Como farei isso?
# Filtrarei o DataFrame filtrado acima para aqueles registros cujo 'total_cases' é igual ou maior que 50
# O .copy() no final, garante que o dataframe 'mt_50_total_cases' seja uma cópia do dataframe filtrado, e não um ponteiro.
mt_50_total_cases = mt_1k_infected_master_df[mt_1k_infected_master_df["total_cases"] >= 50].copy()

# Ordene o DataFrame acima por location e date de forma ascendente (crescente)
mt_50_total_cases.sort_values(by=["location", "date"], ascending=[True, True] inplace=True)

In [ ]:
# Verificando se há algo estranho:
mt_50_total_cases.head(15)

In [ ]:
# Estamos indo para a sessão dos plots.
# Aqui definirei em um dicionário a cor da linha de cada um dos países.
country_colors = {
    "Brazil": "#4AE8AE",        #verde
    "China": "#E84623",         #vermelho
    "Italy": "#E88C2C",         #laranja
    "Spain": "#FF4C8A",         #rosa
    "United States": "#2543E8",  #azul
}

### PLOTTING TIME
- Teremos 4 gráficos
- Para todos os 4, x será o número de dias contados a partir do caso de covid-19 de número 50.
- Quanto ao eixo y:
    - Um será o número de casos totais
    - Um será o número de casos totais com transformação logarítimica
    - Um será o número de mortes totais
    - Um será o número de mortes totais com transformação logarítimica

In [ ]:
# Como usaremos a função 'plot' do 'matplotlib.pyplot' várias vezes, precisaremos usar o 'subplots' a fim de gerar um eixo,
# ou seja, um ax. Isso para, sempre que formos chamar a função plot, atribuirmos que o plot será desenhado no eixo ax.
fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(16, 8))

# Para o Brazil
# Primeira coisa, precisamos filtrar o dataframe 'mt_50_total_cases' - que já havia sido filtrado por 'total_cases' acima -
# por location, como esta primeira linha é para o Brazil, filtraremos location == BR
filtered_by_Brazil_df = mt_50_total_cases[mt_50_total_cases["location"] == "Brazil"]
# Após, pegaremos o número único de datas para o BR e criaremos uma PA com o tamanho do número de datas únicas.
n_unique_dates = filtered_by_Brazil_df.date.nunique()
ind = np.arange(n_unique_dates)
# Agora, pegaremos a coluna total_cases. Se quiséssemos plotar outra coluna, como a 'total_deaths', precisaríamos
# trocar a linha abaixo a fim de selecionar a coluna de nosso interesse.
total_cases = filtered_by_Brazil_df.total_cases.tolist()
# Enfim, plotaremos a linha para o Brazil. Configurando, para esta linha, a label "Brazil" e a cor "#4AE8AE" que é um verde água
ax1.plot(ind, total_cases, ".-", label="Brazil", color="#4AE8AE")
# Repetiremos esse mesmo processo, só que para cada país. Lembrando de plotar a linha sempre no ax1, pois foi pra isso que
# criamos o ax1, para que todas as linhas fossem plotadas na mesma axis.


# Para a China
filtered_by_China_df = mt_50_total_cases[mt_50_total_cases["location"] == "China"]
n_unique_dates = filtered_by_China_df.date.nunique()
ind = np.arange(n_unique_dates)
total_cases = filtered_by_China_df.total_cases.tolist()
ax1.plot(ind, total_cases, ".-", label="China", color="#E84623")


# Para a Itália
filtered_by_Italy_df = mt_50_total_cases[mt_50_total_cases["location"] == "Italy"]
n_unique_dates = filtered_by_Italy_df.date.nunique()
ind = np.arange(n_unique_dates)
total_cases = filtered_by_Italy_df.total_cases.tolist()
ax1.plot(ind, total_cases, ".-", label="Italy", color="#E88C2C")


# Para a Espanha
filtered_by_Spain_df = mt_50_total_cases[mt_50_total_cases["location"] == "Spain"]
n_unique_dates = filtered_by_Spain_df.date.nunique()
ind = np.arange(n_unique_dates)
total_cases = filtered_by_Spain_df.total_cases.tolist()
ax1.plot(ind, total_cases, ".-", label="Spain", color="#FF4C8A")


# Para os EUA
filtered_by_US_df = mt_50_total_cases[mt_50_total_cases["location"] == "United States"]
n_unique_dates = filtered_by_US_df.date.nunique()
ind = np.arange(n_unique_dates)
total_cases = filtered_by_US_df.total_cases.tolist()
ax1.plot(ind, total_cases, ".-", label="US", color="#2543E8")


# Agora, atribuiremos um título
ax1.set_title("Total Cases from 50th Case")
# A posição dos ticks do eixo x
ax1.set_xticks(np.arange(80 + 1))
# O rótulo dos ticks do eixo x
ax1.set_xticklabels([x if x % 2 == 0 else "" for x in np.arange(80 + 1)])
# O rótulo para todo o eixo X
ax1.set_xlabel("Days from 50th Case")
# O rótulo para todo o eixo Y
ax1.set_ylabel("Total Cases")

# Se quisermos transformar o eixo y com log, bastaria usar a linha abaixo que, por ora, está comentada.
#ax1.set_yscale("log")

# Ativaremos o legend para que as labels das linhas sejam mostradas
ax1.legend();

Beleza, utilizamos toda a célula acima para um único gráfico.
O que mais tomou nosso tempo, foi repetir a mesma coisa que fizemos para o Brazil nos outros países. Por isso, um loop facilita nossa vida. Abaixo, temos o processo acima usando o loop e, dessa vez, para os 4 plots que desejávamos plotar.


In [ ]:
# Com plt.subplots, definimos quantos subplots plotaremos de uma vez só. 
# Nesse caso, serão 4.
fig, ax = plt.subplots(4, 1, figsize=(16, 8*4))

# A variável ax criada acima, está com as 4 axis de interesse. 
# Para desmembrar o ax, utilizamos ax.flat.
# Daí, teremos que ax1 será o plot sobre total-cases e ax2 o msm que ax1 só que com log no y
# E ax3 será o plot sobre total_deaths e ax4 o msm que o ax3 só que com log no y
ax1, ax2, ax3, ax4 = ax.flat


# Vamos iterar sobre os países que filtramos. Para cada iteração, a variável 'country' armazenará a string do país em questão
for country in filtered_countries:
    # Filtraremos 'mt_50_total_cases' quando sua location for igual a 'country', ou seja, o país da iteração atual.
    # O dataframe filtrado será armazenado em 'filtered_by_country_df'
    filtered_by_country_df = mt_50_total_cases[mt_50_total_cases["location"] == country]
    
    # Pegaremos o número único de datas daqueles país e faremos uma PA com esse valor. A PA ficará armazenada em ind e será
    # os valores de X para os nossos plots
    n_unique_dates = filtered_by_country_df.date.nunique()
    ind = np.arange(n_unique_dates)
    
    # Pegaremos tanto a coluna 'total_deaths' quanto 'total_cases' para o país em questão do dataframe 
    # criado no começo do loop
    total_deaths = filtered_by_country_df.total_deaths.tolist()
    total_cases = filtered_by_country_df.total_cases.tolist()
    
    # Enfim, plotaremos a linha para o país em cada um dos axis
    # A princípio, ax1 e ax2 serão iguais, assim como ax3 e ax4. Visto que ax2 e ax4 são, respectivamente,
    # a mesma coisa que ax1 e ax3 com a pequena diferença da transformação logarítimica no eixo Y.
    # Essa transformação será feita após todas as linhas serem plotadas, pois não é necessário fazer a transformação
    # para toda iteração, apenas uma única vez.
    ax1.plot(ind, total_cases, ".-", label=country, color=country_colors[country])
    ax2.plot(ind, total_cases, ".-", label=country, color=country_colors[country])
    
    ax3.plot(ind, total_deaths, ".-", label=country, color=country_colors[country])
    ax4.plot(ind, total_deaths, ".-", label=country, color=country_colors[country])

    
# Transformando o eixo Y do ax2 e ax4 com log
ax2.set_yscale("log")
ax4.set_yscale("log")

# Adicionando título, ticks e labels para todos os axs
ax1.set_title("Total Cases from 50th Case")
ax1.set_xticks(np.arange(80 + 1))
ax1.set_xticklabels([x if x % 2 == 0 else "" for x in np.arange(80 + 1)])
ax1.set_xlabel("Days from 50th Case")
ax1.set_ylabel("Total Cases")

ax2.set_title("Total Cases (log) from 50th Case")
ax2.set_xticks(np.arange(80 + 1))
ax2.set_xticklabels([x if x % 2 == 0 else "" for x in np.arange(80 + 1)])
ax2.set_xlabel("Days from 50th Case")
ax2.set_ylabel("Total Cases (log transformation)")

ax3.set_title("Total Deaths from 50th Case")
ax3.set_xticks(np.arange(80 + 1))
ax3.set_xticklabels([x if x % 2 == 0 else "" for x in np.arange(80 + 1)])
ax3.set_xlabel("Days from 50th Case")
ax3.set_ylabel("Total Deaths")

ax4.set_title("Total Deaths from 50th Case")
ax4.set_xticks(np.arange(80 + 1))
ax4.set_xticklabels([x if x % 2 == 0 else "" for x in np.arange(80 + 1)])
ax4.set_xlabel("Days from 50th Case")
ax4.set_ylabel("Total Deaths (log transformation)")


# Aqui, eu quis fazer graça. Basicamente, no axis 3 (ax3), plotar uma linha horizontal na mesma altura
# do total de número de mortes da china. Assim, conseguiremos ver quando um país já ultrapassou o total de mortes da china.
# Primeiro, filtramos o dataframe original para location == China e pegamos o máximo de total_deaths. Armazenamos
# esse valor em 'china_max_deaths'
china_max_deaths = df[df["location"] == "China"].total_deaths.max()
# Após, plotamos a linha horiozntal, na altura de "china_max_deaths", além disso, adicionei o estilo de linha tracejado
ax3.axhline(china_max_deaths, linestyle="--", linewidth=1, color="black")


ax1.legend()
ax2.legend()
ax3.legend()
ax4.legend();

### CONSIDERAÇÕES
- Nesses primeiros dias após o 50º caso, a evolução de casos de Covid-19 no Brasil é semelhante à da Espanha, mais agressiva que a da China e dos Eua. Nesse período, a evolução não segue o ritmo da Itália.
- Como o Brasil segue a curva da Espanha quanto ao número de casos totais e verificamos que a Espanha supera o número de casos da Itália após o 16º dia após o 50º caso, pode-se dizer que o Brasil tende também, a superar a evolução da Itália na próxima semana, comparado ao mesmo período que a Itália se encontrava.
- Quanto ao número de mortes, a Itália atingiu o número de mortes que a China atingiu em 80 dias em 1/4 do tempo. Para piorar, a curva da Itália não leva a crer que sua situação esteja controlada como a da China. A Espanha tende a ultrapassar o total de mortes que a Itália apresentava neste mesmo período nos próximos dias. O Brasil acompanha a evolução do quadro de mortes da Espanha nessas 2 primeiras semanas após o 50º caso.
- Podemos dizer que a situação brasileira está pior ou melhor que esses casos? Acredito que não exatamente, eu buscaria dados sobre o número de casos suspeitos e o números de casos suspeitos que são testados para a Covid-19 em cada um desses países. Por síndrome do vira-lato, acredito que testemos menos pessoas que a Espanha e, por isso, podemos ter números mais expressivos que a Itália a Espanha nas próximas semanas comparado com o mesmo período que esses países se encontravam.

### DESAFIO
Plotar a evolução da proporção total_deaths / total_cases para esses países a partir do 50º caso.

In [ ]:
# Para ajudá-las, vou criar a coluna death_prop no dataframe original:

df["death_prop"] = df["total_deaths"] / df["total_cases"]